In [ ]:
# @title Setup
!git clone https://github.com/wandb/edu.git
%cd edu/rag-advanced
!pip install -qqq -r requirements.txt
%cd notebooks

import nltk

nltk.download("wordnet")

Cloning into 'edu'...
remote: Enumerating objects: 4510, done.
remote: Counting objects: 100% (1412/1412), done.
remote: Compressing objects: 100% (532/532), done.
remote: Total 4510 (delta 1076), reused 1042 (delta 877), pack-reused 3098 (from 1)
Receiving objects: 100% (4510/4510), 28.01 MiB | 8.41 MiB/s, done.
Resolving deltas: 100% (2471/2471), done.
/content/edu/rag-advanced
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 9.6 MB/s eta 0:00:0

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass("Please enter your COHERE_API_KEY")

Please enter your COHERE_API_KEY··········


In [ ]:
import asyncio

import nest_asyncio

nest_asyncio.apply()

import weave

from scripts.utils import display_source

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
WANDB_PROJECT = "rag-course"
weave_client = weave.init(WANDB_PROJECT)

Please login to Weights & Biases (https://wandb.ai/) to continue:


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Logged in as Weights & Biases user: shikhar_dave.
View Weave data at https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/weave


In [ ]:
# Easy eval dataset with 20 samples.
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:Qj4IFICc2EbdXu5A5UuhkPiWgxM1GvJMIvXEyv1DYnM"
).get()

print("Number of evaluation samples: ", len(eval_dataset.rows))

Number of evaluation samples:  20


In [ ]:
dict(eval_dataset.rows[0])

{'question': 'How can I access the run object from the Lightning WandBLogger function?',
 'answer': "In PyTorch Lightning, the `WandbLogger` is used to log metrics, model weights, and other data to Weights & Biases during training. To access the `wandb.Run` object from within a `LightningModule` when using `WandbLogger`, you can use the `Trainer.logger.experiment` attribute. This attribute provides direct access to the underlying `wandb.Run` object, allowing you to interact with the Weights & Biases API directly.\n\nHere's how you can access the `wandb.Run` object using `WandbLogger` in PyTorch Lightning:\n\n```python\nfrom pytorch_lightning import Trainer, LightningModule\nfrom pytorch_lightning.loggers import WandbLogger\n\nclass MyModel(LightningModule):\n    def training_step(self, batch, batch_idx):\n        # Your training logic here\n        loss = ...\n\n        # Log metrics\n        self.log('train_loss', loss)\n\n        # Access the wandb.Run object\n        run = self.trai

In [ ]:
# Reload the data from Chapter 1
chunked_data = weave.ref(
    "weave:///rag-course/rag-course/object/chunked_data:Lt6M8qCUICD1JZTlMYzuDLTVtvFYESxvj3tcAIoPrtE"
).get()
chunked_data = weave.ref("chunked_data:v0").get()
print("Number of chunked data: ", len(chunked_data.rows))
chunked_data.rows[:2]

Number of chunked data:  1073


[WeaveDict({'content': '--- description: Log and visualize data without a W&B account displayed_sidebar: default --- # Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with `wandb.init(`**`anonymous="allow"`**`)` :::info **Publishing a paper?** Please [cite W&B](https://docs.wandb.ai/company/academics#bibtex-citation), and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com. ::: ### How does someone without an account see results? If someone runs your script and you have to set `anonymous="allow"`: 1. **Auto-create temporary account:** W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. 2. **Log r

In [ ]:
from scripts.retriever import TFIDFRetriever

display_source(TFIDFRetriever)

retriever = TFIDFRetriever()
retriever.index_data(list(map(dict, chunked_data.rows)))

   1 class TFIDFRetriever(weave.Model):                                                                            
   2     """                                                                                                       
   3     A retriever model that uses TF-IDF for indexing and searching documents.                                  
   4                                                                                                               
   5     Attributes:                                                                                               
   6         vectorizer (TfidfVectorizer): The TF-IDF vectorizer.                                                  
   7         index (list): The indexed data.                                                                       
   8         data (list): The data to be indexed.                                                                  
   9     """                                                                                                       
  10                                                                                                               
  11     vectorizer: TfidfVectorizer = TfidfVectorizer()                                                           
  12     index: list = None                                                                                        
  13     data: list = None                                                                                         
  14                                                                                                               
  15     def index_data(self, data):                                                                               
  16         """                                                                                                   
  17         Indexes the provided data using TF-IDF.                                                               
  18                                                                                                               
  19         Args:                                                                                                 
  20             data (list): A list of documents to be indexed. Each document should be a dictionary              
  21                          containing a key 'cleaned_content' with the text to be indexed.                      
  22         """                                                                                                   
  23         self.data = data                                                                                      
  24         docs = [doc["cleaned_content"] for doc in data]                                                       
  25         self.index = self.vectorizer.fit_transform(docs)                                                      
  26                                                                                                               
  27     @weave.op()                                                                                               
  28     def search(self, query, k=5):                                                                             
  29         """                                                                                                   
  30         Searches the indexed data for the given query using cosine similarity.                                
  31                                                                                                               
  32         Args:                                                                                                 
  33             query (str): The search query.                                                                    
  34             k (int): The number of top results to return. Default is 5.                                       
  35                                                    

In [ ]:
from scripts.retrieval_metrics import IR_METRICS
from scripts.utils import display_source

In [ ]:
for scorer in IR_METRICS:
    display_source(scorer)

   1 @weave.op                                                                                                     
   2 def compute_hit_rate(                                                                                         
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the hit rate (precision) for a single query.                                                    
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The hit rate (precision).                                                                      
  19                                                                                                               
  20     The hit rate (precision) measures the proportion of retrieved documents that are relevant.                
  21                                                                                                               
  22     This metric is useful for assessing the accuracy of the retrieval system by determining the relevance of t
  23     retrieved documents.                                                                                      
  24     """                                                                                                       
  25     search_results = [doc["source"] for doc in model_output]                                                  
  26     relevant_sources = [                                                                                      
  27         context["source"] for context in contexts if context["relevance"] != 0                                
  28     ]                                                                                                         
  29                                                                                                               
  30     # Calculate the number of relevant documents retrieved                                                    
  31     relevant_retrieved = sum(                                                                                 
  32         1 for source in search_results if source in relevant_sources                                          
  33     )                                                                                                         
  34                                                                                                               
  35     # Calculate the hit rate (precision)           

   1 @weave.op                                                                                                     
   2 def compute_mrr(                                                                                              
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Mean Reciprocal Rank (MRR) for a single query.                                              
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The MRR score for the given query.                                                             
  19                                                                                                               
  20     MRR measures the rank of the first relevant document in the result list.                                  
  21                                                                                                               
  22     If no relevant document is found, MRR is 0.                                                               
  23                                                                                                               
  24     This metric is useful for evaluating systems where there is typically one relevant document               
  25     and the user is interested in finding that document quickly.                                              
  26     """                                                                                                       
  27     relevant_sources = [                                                                                      
  28         context["source"] for context in contexts if context["relevance"] != 0                                
  29     ]                                                                                                         
  30                                                                                                               
  31     mrr_score = 0                                                                                             
  32     for rank, result in enumerate(model_output, 1):                                                           
  33         if result["source"] in relevant_sources:                                                              
  34             mrr_score += 1 / rank                                                                             
  35                                                    

   1 @weave.op                                                                                                     
   2 def compute_ndcg(                                                                                             
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Normalized Discounted Cumulative Gain (NDCG) for a single query.                            
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  13             Each dictionary contains:                                                                         
  14                 - 'source': A unique identifier for the relevant document.                                    
  15                 - 'relevance': The relevance score of the document (0, 1, or 2).                              
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The NDCG score for the given query.                                                            
  19     """                                                                                                       
  20     relevance_map = {context["source"]: context["relevance"] for context in contexts}                         
  21                                                                                                               
  22     dcg = 0.0                                                                                                 
  23     idcg = 0.0                                                                                                
  24                                                                                                               
  25     for i, result in enumerate(model_output):                                                                 
  26         rel = relevance_map.get(result["source"], 0)                                                          
  27         dcg += (2**rel - 1) / np.log2(i + 2)                                                                  
  28                                                                                                               
  29     sorted_relevances = sorted(                                                                               
  30         [context["relevance"] for context in contexts], reverse=True                                          
  31     )                                                                                                         
  32     for i, rel in enumerate(sorted_relevances):                                                               
  33         idcg += (2**rel - 1) / np.log2(i + 2)                                                                 
  34                                                                                                               
  35     [38;

   1 @weave.op                                                                                                     
   2 def compute_map(                                                                                              
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Mean Average Precision (MAP) for a single query.                                            
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The MAP score for the given query.                                                             
  19                                                                                                               
  20     MAP provides a single-figure measure of quality across recall levels.                                     
  21     For a single query, it's equivalent to the Average Precision (AP).                                        
  22                                                                                                               
  23                                                                                                               
  24     Where:                                                                                                    
  25     - n is the number of retrieved documents                                                                  
  26     - P(k) is the precision at cut-off k in the list                                                          
  27     - rel(k) is an indicator function: 1 if the item at rank k is relevant, 0 otherwise                       
  28     MAP considers both precision and recall, as well as the ranking of relevant documents.                    
  29                                                                                                               
  30     """                                                                                                       
  31     relevant_sources = {                                                                                      
  32         context["source"] for context in contexts if context["relevance"] != 0                                
  33     }                                                                                                         
  34                                                                                                               
  35     num_relevant = 0                               

   1 @weave.op                                                                                                     
   2 def compute_precision(                                                                                        
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Precision for a single query.                                                               
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The Precision score for the given query.                                                       
  19                                                                                                               
  20     Precision measures the proportion of retrieved documents that are relevant.                               
  21     """                                                                                                       
  22     relevant_sources = {                                                                                      
  23         context["source"] for context in contexts if context["relevance"] != 0                                
  24     }                                                                                                         
  25     retrieved_sources = {result["source"] for result in model_output}                                         
  26                                                                                                               
  27     relevant_retrieved = relevant_sources & retrieved_sources                                                 
  28                                                                                                               
  29     precision = (                                                                                             
  30         len(relevant_retrieved) / len(retrieved_sources) if retrieved_sources else 0.0                        
  31     )                                                                                                         
  32     return precision                                                                                          
  33                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_recall(                                                                                           
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the Recall for a single query.                                                                  
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The Recall score for the given query.                                                          
  19                                                                                                               
  20     Recall measures the proportion of relevant documents that are retrieved.                                  
  21     """                                                                                                       
  22     relevant_sources = {                                                                                      
  23         context["source"] for context in contexts if context["relevance"] != 0                                
  24     }                                                                                                         
  25     retrieved_sources = {result["source"] for result in model_output}                                         
  26                                                                                                               
  27     relevant_retrieved = relevant_sources & retrieved_sources                                                 
  28                                                                                                               
  29     recall = (                                                                                                
  30         len(relevant_retrieved) / len(relevant_sources) if relevant_sources else 0.0                          
  31     )                                                                                                         
  32     return recall                                                                                             
  33                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_f1_score(                                                                                         
   3     model_output: List[Dict[str, Any]], contexts: List[Dict[str, Any]]                                        
   4 ) -> float:                                                                                                   
   5     """                                                                                                       
   6     Calculate the F1-Score for a single query.                                                                
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10             Each dictionary contains:                                                                         
  11                 - 'source': A unique identifier for the document.                                             
  12                 - 'score': The relevance score of the document.                                               
  13         contexts (List[Dict[str, Any]]): A list of dictionaries representing the relevant contexts.           
  14             Each dictionary contains:                                                                         
  15                 - 'source': A unique identifier for the relevant document.                                    
  16                                                                                                               
  17     Returns:                                                                                                  
  18         float: The F1-Score for the given query.                                                              
  19                                                                                                               
  20     F1-Score is the harmonic mean of Precision and Recall.                                                    
  21     """                                                                                                       
  22     precision = compute_precision(model_output, contexts)                                                     
  23     recall = compute_recall(model_output, contexts)                                                           
  24                                                                                                               
  25     if precision + recall == 0:                                                                               
  26         return 0.0                                                                                            
  27                                                                                                               
  28     f1_score = 2 * (precision * recall) / (precision + recall)                                                
  29     return f1_score                                                                                           
  30                                                                                                               

In [ ]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=IR_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k": 5},
)

retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

Evaluated 1 of 20 examples

Evaluated 2 of 20 examples

Evaluated 3 of 20 examples

Evaluated 4 of 20 examples

Evaluated 5 of 20 examples

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'compute_hit_rate': {'mean': 0.37},
    'compute_mrr': {'mean': 0.18533333333333332},
    'compute_ndcg': {'mean': 0.2455147810080979},
    'compute_map': {'mean': 0.45282142857142854},
    'compute_precision': {'mean': 0.31500000000000006},
    'compute_recall': {'mean': 0.29464285714285715},
    'compute_f1_score': {'mean': 0.29965007215007217},
    'model_latency': {'mean': 3.6908488869667053}
}

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/019275da-6a1f-76b3-8484-27baf231b941


In [ ]:
from scripts.retrieval_metrics import LLM_METRICS

for metric in LLM_METRICS:
    display_source(metric)

   1 @weave.op                                                                                                     
   2 async def llm_retrieval_scorer(                                                                               
   3     model_output: List[Dict[str, Any]], question: str                                                         
   4 ) -> Dict[str, float]:                                                                                        
   5     """                                                                                                       
   6     Evaluate the retrieval results using a language model and compute relevance scores.                       
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (List[Dict[str, Any]]): The list of retrieved documents from the model.                  
  10         question (str): The query or question for which the retrieval is being evaluated.                     
  11                                                                                                               
  12     Returns:                                                                                                  
  13         Dict[str, float]: A dictionary containing the mean relevance score and the relevance rank score.      
  14     """                                                                                                       
  15     scores = await evaluate_retrieval_with_llm(question, model_output)                                        
  16     relevance_scores = [item["relevance"] for item in scores["final_scores"]]                                 
  17     mean_relevance = sum(relevance_scores) / len(model_output)                                                
  18     rank_score = compute_rank_score(relevance_scores)                                                         
  19     return {"relevance": mean_relevance, "relevance_rank_score": rank_score}                                  
  20                                                                                                               

In [ ]:
retrieval_evaluation = weave.Evaluation(
    name="LLM_Judge_Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=LLM_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"], "k": 5},
)
retrieval_scores = asyncio.run(retrieval_evaluation.evaluate(retriever))

Predict and score failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 287, in eval_example
    eval_row = await self.predict_and_score(model, example)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 459, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 287, in _call_async
    return handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 285, in _call_async
    res = await func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 224, in predict_and_score
    result = await async_call(score_fn, **score_args)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 459, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/o

Predict and score failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 287, in eval_example
    eval_row = await self.predict_and_score(model, example)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 459, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 287, in _call_async
    return handle_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 285, in _call_async
    res = await func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 224, in predict_and_score
    result = await async_call(score_fn, **score_args)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 459, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/o

Predict and score failed

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{
    'llm_retrieval_scorer': {
        'relevance': {'mean': 0.6444444444444444},
        'relevance_rank_score': {'mean': 0.6222222222222222}
    },
    'model_latency': {'mean': 3.8790869447920056}
}

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/019275e0-9973-7083-8666-71ec7bd69357


In [ ]:
from scripts.rag_pipeline import SimpleRAGPipeline
from scripts.response_generator import SimpleResponseGenerator

INITIAL_PROMPT = open("prompts/initial_system.txt", "r").read()
response_generator = SimpleResponseGenerator(model="command-r", prompt=INITIAL_PROMPT)
rag_pipeline = SimpleRAGPipeline(
    retriever=retriever, response_generator=response_generator, top_k=5
)

In [ ]:
from scripts.response_metrics import NLP_METRICS

for scorer in NLP_METRICS:
    display_source(scorer)

   1 @weave.op                                                                                                     
   2 def compute_diff(model_output: str, answer: str) -> float:                                                    
   3     """                                                                                                       
   4     Compute the similarity ratio between the normalized model output and the expected answer.                 
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The output generated by the model.                                                
   8         answer (str): The expected answer.                                                                    
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The similarity ratio between the normalized model output and the expected answer.              
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     return difflib.SequenceMatcher(None, norm_output, norm_answer).ratio()                                    
  16                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_levenshtein(model_output: str, answer: str) -> float:                                             
   3     """                                                                                                       
   4     Compute the Levenshtein ratio between the normalized model output and the answer.                         
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The output generated by the model.                                                
   8         answer (str): The expected answer.                                                                    
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The Levenshtein ratio between the normalized model output and the answer.                      
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     return Levenshtein.ratio(norm_output, norm_answer)                                                        
  16                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_rouge(model_output: str, answer: str) -> float:                                                   
   3     """                                                                                                       
   4     Compute the ROUGE-L F1 score between the normalized model output and the reference answer.                
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The model's generated output.                                                     
   8         answer (str): The reference answer.                                                                   
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The ROUGE-L F1 score.                                                                          
  12     """                                                                                                       
  13     norm_output = normalize_text(model_output)                                                                
  14     norm_answer = normalize_text(answer)                                                                      
  15     rouge = Rouge(metrics=["rouge-l"], stats="f")                                                             
  16     scores = rouge.get_scores(norm_output, norm_answer)                                                       
  17     return scores[0]["rouge-l"]["f"]                                                                          
  18                                                                                                               

   1 @weave.op                                                                                                     
   2 def compute_bleu(model_output: str, answer: str) -> float:                                                    
   3     """                                                                                                       
   4     Compute the BLEU score between the normalized model output and the reference answer.                      
   5                                                                                                               
   6     Args:                                                                                                     
   7         model_output (str): The generated output from the model.                                              
   8         answer (str): The reference answer.                                                                   
   9                                                                                                               
  10     Returns:                                                                                                  
  11         float: The BLEU score between the normalized model output and the reference answer.                   
  12     """                                                                                                       
  13     chencherry = SmoothingFunction()                                                                          
  14     smoothing_function = chencherry.method2                                                                   
  15                                                                                                               
  16     norm_output = normalize_text(model_output)                                                                
  17     norm_answer = normalize_text(answer)                                                                      
  18     reference = word_tokenize(norm_answer)                                                                    
  19     candidate = word_tokenize(norm_output)                                                                    
  20     score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)                      
  21     return score                                                                                              
  22                                                                                                               

In [ ]:
response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=NLP_METRICS[:-1],
    preprocess_model_input=lambda x: {"query": x["question"]},
)

response_scores = asyncio.run(response_evaluations.evaluate(rag_pipeline))

NameError: name 'response_evaluations' is not defined

In [ ]:
from scripts.response_metrics import LLM_METRICS

for metric in LLM_METRICS:
    display_source(metric)

   1 @weave.op                                                                                                     
   2 async def llm_response_scorer(                                                                                
   3     model_output: str, question: str, answer: str                                                             
   4 ) -> Dict[str, Any]:                                                                                          
   5     """                                                                                                       
   6     Evaluate the correctness of the model output using a language model judge.                                
   7                                                                                                               
   8     Args:                                                                                                     
   9         model_output (str): The output generated by the model.                                                
  10         question (str): The question posed to the model.                                                      
  11         answer (str): The reference answer.                                                                   
  12                                                                                                               
  13     Returns:                                                                                                  
  14         Dict[str, Any]: A dictionary containing the final score and a boolean indicating correctness.         
  15     """                                                                                                       
  16     evaluation_result = await evaluate_correctness_using_llm_judge(                                           
  17         question, answer, model_output                                                                        
  18     )                                                                                                         
  19                                                                                                               
  20     return {                                                                                                  
  21         "score": evaluation_result["final_score"],                                                            
  22         "correct": evaluation_result["decision"] == "correct",                                                
  23     }                                                                                                         
  24                                                                                                               

In [ ]:
correctness_evaluations = weave.Evaluation(
    name="Correctness_Evaluation",
    dataset=eval_dataset,
    scorers=LLM_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"]},
)

response_scores = asyncio.run(correctness_evaluations.evaluate(rag_pipeline))

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 167, in predict_and_score
    model_output = await async_call(model_predict, **model_predict_args)
  File "/usr/lib/python3.10/asyncio/threads.py", line 25, in to_thread
    return await loop.run_in_executor(None, func_call)
  File "/usr/lib/python3.10/asyncio/futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup
    future.result()
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 482, in wrapper
    res, _ = _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3

model_output failed

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/weave/flow/eval.py", line 167, in predict_and_score
    model_output = await async_call(model_predict, **model_predict_args)
  File "/usr/lib/python3.10/asyncio/threads.py", line 25, in to_thread
    return await loop.run_in_executor(None, func_call)
  File "/usr/lib/python3.10/asyncio/futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup
    future.result()
  File "/usr/lib/python3.10/asyncio/futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.10/dist-packages/weave/trace/op.py", line 482, in wrapper
    res, _ = _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
  File "/usr/local/lib/python3

model_output failed

...

Evaluated 18 of 20 examples

Evaluated 19 of 20 examples

Evaluated 20 of 20 examples

Evaluation summary
{'llm_response_scorer': None}

🍩 https://wandb.ai/shikhar_dave-iit-jodhpur/rag-course/r/call/019275e2-0d8e-7980-ad56-5a676af15ce1
